In [83]:
import requests
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
from clickhouse_connect import get_client
from clickhouse_driver import Client
import time
from typing import List, Dict

load_dotenv()

True

In [84]:
KeySmart = os.getenv("KeySmart")
password = os.getenv('ClickHouse')

In [132]:
class WildberriesAPI:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://statistics-api.wildberries.ru/api/v5/supplier/reportDetailByPeriod"
        self.last_request = 0
    
    def _wait_if_needed(self):
        """Wait if less than 60 seconds since last request"""
        elapsed = time.time() - self.last_request
        if elapsed < 60:
            time.sleep(60 - elapsed)
    
    def get_report(self, date_from: str, date_to: str, limit: int = 100000) -> List[Dict]:
        """Fetch report with automatic pagination"""
        results = []
        rrdid = 0
        
        while True:
            self._wait_if_needed()
            
            params = {
                "dateFrom": date_from,
                "dateTo": date_to,
                "limit": min(limit, 100000),
                "rrdid": rrdid
            }
            
            response = requests.get(
                self.base_url,
                headers={"Authorization": self.api_key},
                params=params
            )
            
            self.last_request = time.time()
            
            if not response.ok:
                print(f"Error: {response.status_code}")
                break
                
            data = response.json()
            if not data:
                break
                
            results.extend(data)
            rrdid = data[-1].get("rrd_id", 0)
            
            if len(data) < limit:
                break
        
        return results


# Example usage
if __name__ == "__main__":
    api = WildberriesAPI(api_key=KeySmart)
    
    report = api.get_report(
        date_from="2025-01-01",
        date_to="2025-04-13"
    )
    
    print(f"Got {len(report)} records")
    if report:
        print("First record:", report[0])
        
    # Convert to pandas DataFrame
    df = pd.DataFrame(report)

    # Basic data cleaning
    df = df.applymap(lambda x: None if x == '' else x)  # Replace empty strings with None



Got 30785 records
First record: {'realizationreport_id': 293925472, 'date_from': '2024-12-30', 'date_to': '2025-01-05', 'create_dt': '2025-01-06', 'currency_name': 'KZT', 'suppliercontract_code': None, 'rrd_id': 2890007707632, 'gi_id': 25506386, 'dlv_prc': 1.95, 'fix_tariff_date_from': '2024-12-16T21:00:00Z', 'fix_tariff_date_to': '2025-02-14T21:00:00Z', 'subject_name': 'Бритвы электрические', 'nm_id': 205352338, 'brand_name': 'Braun', 'sa_name': 'б0058410', 'ts_name': '0', 'barcode': '4210201432593', 'doc_type_name': '', 'quantity': 0, 'retail_price': 0, 'retail_amount': 0, 'sale_percent': 0, 'commission_percent': 0, 'office_name': 'Казань', 'supplier_oper_name': 'Логистика', 'order_dt': '2024-12-19T19:38:45Z', 'sale_dt': '2025-01-01T17:13:20Z', 'rr_dt': '2025-01-01', 'shk_id': 26453644481, 'retail_price_withdisc_rub': 0, 'delivery_amount': 1, 'return_amount': 0, 'delivery_rub': 473.51, 'gi_box_type_name': 'Микс', 'product_discount_for_report': 0, 'supplier_promo': 0, 'rid': 0, 'ppvz_

C:\Users\User\AppData\Local\Temp\ipykernel_14532\1029100513.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if x == '' else x)  # Replace empty strings with None


In [133]:
df

,realizationreport_id,date_from,date_to,create_dt,currency_name,suppliercontract_code,rrd_id,gi_id,dlv_prc,fix_tariff_date_from,...,rebill_logistic_cost,storage_fee,deduction,acceptance,assembly_id,srid,report_type,is_legal_entity,trbx_id,rebill_logistic_org
0,293925472,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707632,25506386,1.95,2024-12-16T21:00:00Z,...,0.00,0.00,0.0,0.0,0,371b67061407497ea211be3c07962439,2,False,None,NaN
1,293925472,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707633,25506386,1.95,2024-12-16T21:00:00Z,...,0.00,0.00,0.0,0.0,0,371b67061407497ea211be3c07962439,2,False,None,NaN
2,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707634,0,0.00,None,...,8.32,0.00,0.0,0.0,0,15376827102385309.0.0,1,False,None,ИП Меняйло Ольга Ивановна
3,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707635,0,0.00,None,...,66.36,0.00,0.0,0.0,0,61948635601833665.0.0,1,False,None,Индивидуальный предприниматель Савон Дмитрий...
4,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707636,0,0.00,None,...,26.11,0.00,0.0,0.0,0,20087858102595480.0.0,1,False,None,ИП Николаенко Василий Андреевич
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30780,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569248,0,0.00,None,...,54.99,0.00,0.0,0.0,0,3502224,1,False,None,NaN
30781,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569249,0,0.00,None,...,44.45,0.00,0.0,0.0,0,3502229,1,False,None,NaN
30782,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569250,0,0.00,None,...,44.45,0.00,0.0,0.0,0,3502230,1,False,None,NaN
30783,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569251,0,0.00,None,...,57.37,0.00,0.0,0.0,0,3502232,1,False,None,NaN


In [134]:
# Add missing columns with default values
if 'load_dt' not in df.columns:
    df['load_dt'] = pd.Timestamp.now()
if 'source' not in df.columns:
    df['source'] = "WB-Realization-API"
    
df

,realizationreport_id,date_from,date_to,create_dt,currency_name,suppliercontract_code,rrd_id,gi_id,dlv_prc,fix_tariff_date_from,...,deduction,acceptance,assembly_id,srid,report_type,is_legal_entity,trbx_id,rebill_logistic_org,load_dt,source
0,293925472,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707632,25506386,1.95,2024-12-16T21:00:00Z,...,0.0,0.0,0,371b67061407497ea211be3c07962439,2,False,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
1,293925472,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707633,25506386,1.95,2024-12-16T21:00:00Z,...,0.0,0.0,0,371b67061407497ea211be3c07962439,2,False,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
2,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707634,0,0.00,None,...,0.0,0.0,0,15376827102385309.0.0,1,False,None,ИП Меняйло Ольга Ивановна,2025-04-15 18:25:08.507613,WB-Realization-API
3,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707635,0,0.00,None,...,0.0,0.0,0,61948635601833665.0.0,1,False,None,Индивидуальный предприниматель Савон Дмитрий...,2025-04-15 18:25:08.507613,WB-Realization-API
4,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707636,0,0.00,None,...,0.0,0.0,0,20087858102595480.0.0,1,False,None,ИП Николаенко Василий Андреевич,2025-04-15 18:25:08.507613,WB-Realization-API
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30780,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569248,0,0.00,None,...,0.0,0.0,0,3502224,1,False,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
30781,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569249,0,0.00,None,...,0.0,0.0,0,3502229,1,False,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
30782,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569250,0,0.00,None,...,0.0,0.0,0,3502230,1,False,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
30783,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569251,0,0.00,None,...,0.0,0.0,0,3502232,1,False,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API


In [135]:
import logging

df = df.copy()

# Ensure the DataFrame has the correct columns
columns = [
    'realizationreport_id', 'date_from', 'date_to', 'create_dt', 'currency_name',
    'suppliercontract_code', 'rrd_id', 'gi_id', 'dlv_prc', 'fix_tariff_date_from',
    'fix_tariff_date_to', 'subject_name', 'nm_id', 'brand_name', 'sa_name', 'ts_name',
    'barcode', 'doc_type_name', 'quantity', 'retail_price', 'retail_amount',
    'sale_percent', 'commission_percent', 'office_name', 'supplier_oper_name',
    'order_dt', 'sale_dt', 'rr_dt', 'shk_id', 'retail_price_withdisc_rub',
    'delivery_amount', 'return_amount', 'delivery_rub', 'gi_box_type_name',
    'product_discount_for_report', 'supplier_promo', 'rid', 'ppvz_spp_prc',
    'ppvz_kvw_prc_base', 'ppvz_kvw_prc', 'sup_rating_prc_up', 'is_kgvp_v2',
    'ppvz_sales_commission', 'ppvz_for_pay', 'ppvz_reward', 'acquiring_fee',
    'acquiring_percent', 'payment_processing', 'acquiring_bank', 'ppvz_vw',
    'ppvz_vw_nds', 'ppvz_office_name', 'ppvz_office_id', 'ppvz_supplier_id',
    'ppvz_supplier_name', 'ppvz_inn', 'declaration_number', 'bonus_type_name',
    'sticker_id', 'site_country', 'srv_dbs', 'penalty', 'additional_payment',
    'rebill_logistic_cost', 'storage_fee', 'deduction', 'acceptance', 'assembly_id',
    'srid', 'report_type', 'is_legal_entity', 'trbx_id', 'rebill_logistic_org',
    'load_dt', 'source'
]


# Convert date columns to datetime and handle NaT values
date_cols = ['date_from', 'date_to', 'create_dt', 'fix_tariff_date_from',
                'fix_tariff_date_to', 'order_dt', 'sale_dt', 'rr_dt', 'load_dt']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.tz_localize(None)
        df[col] = df[col].where(pd.notnull(df[col]), None)  # Replace NaT with None

# 3. Convert boolean columns to integers
df['srv_dbs'] = df['srv_dbs'].astype(int)
df['is_legal_entity'] = df['is_legal_entity'].astype(int)

# 4. Handle nullable string columns
nullable_cols = [
    'suppliercontract_code', 'payment_processing', 'acquiring_bank',
    'ppvz_office_name', 'ppvz_supplier_name', 'ppvz_inn',
    'declaration_number', 'trbx_id', 'rebill_logistic_org'
]

for col in nullable_cols:
    df[col] = df[col].replace('', None)

# Reorder columns to match the expected order
data_organized = df[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in data_organized.to_numpy()]

In [136]:
data

[(293925472,
  Timestamp('2024-12-30 00:00:00'),
  Timestamp('2025-01-05 00:00:00'),
  Timestamp('2025-01-06 00:00:00'),
  'KZT',
  None,
  2890007707632,
  25506386,
  1.95,
  Timestamp('2024-12-16 21:00:00'),
  Timestamp('2025-02-14 21:00:00'),
  'Бритвы электрические',
  205352338,
  'Braun',
  'б0058410',
  '0',
  '4210201432593',
  None,
  0,
  0.0,
  0.0,
  0,
  0.0,
  'Казань',
  'Логистика',
  Timestamp('2024-12-19 19:38:45'),
  Timestamp('2025-01-01 17:13:20'),
  Timestamp('2025-01-01 00:00:00'),
  26453644481,
  0.0,
  1,
  0,
  473.51,
  'Микс',
  0,
  0,
  0,
  0.0,
  0.0,
  0.0,
  0,
  0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  None,
  None,
  0.0,
  0.0,
  None,
  209438,
  0,
  None,
  None,
  None,
  'К клиенту при отмене',
  '0',
  'Россия',
  0,
  0.0,
  0,
  0.0,
  0.0,
  0.0,
  0.0,
  0,
  '371b67061407497ea211be3c07962439',
  2,
  0,
  None,
  nan,
  Timestamp('2025-04-15 18:25:08.507613'),
  'WB-Realization-API'),
 (293925472,
  Timestamp('2024-12-30 00:00:00'),
  Ti

In [137]:
data_organized

,realizationreport_id,date_from,date_to,create_dt,currency_name,suppliercontract_code,rrd_id,gi_id,dlv_prc,fix_tariff_date_from,...,deduction,acceptance,assembly_id,srid,report_type,is_legal_entity,trbx_id,rebill_logistic_org,load_dt,source
0,293925472,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707632,25506386,1.95,2024-12-16 21:00:00,...,0.0,0.0,0,371b67061407497ea211be3c07962439,2,0,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
1,293925472,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707633,25506386,1.95,2024-12-16 21:00:00,...,0.0,0.0,0,371b67061407497ea211be3c07962439,2,0,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
2,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707634,0,0.00,NaT,...,0.0,0.0,0,15376827102385309.0.0,1,0,None,ИП Меняйло Ольга Ивановна,2025-04-15 18:25:08.507613,WB-Realization-API
3,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707635,0,0.00,NaT,...,0.0,0.0,0,61948635601833665.0.0,1,0,None,Индивидуальный предприниматель Савон Дмитрий...,2025-04-15 18:25:08.507613,WB-Realization-API
4,293925474,2024-12-30,2025-01-05,2025-01-06,KZT,None,2890007707636,0,0.00,NaT,...,0.0,0.0,0,20087858102595480.0.0,1,0,None,ИП Николаенко Василий Андреевич,2025-04-15 18:25:08.507613,WB-Realization-API
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30780,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569248,0,0.00,NaT,...,0.0,0.0,0,3502224,1,0,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
30781,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569249,0,0.00,NaT,...,0.0,0.0,0,3502229,1,0,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
30782,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569250,0,0.00,NaT,...,0.0,0.0,0,3502230,1,0,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API
30783,332475507,2025-04-07,2025-04-13,2025-04-14,KZT,None,2914046569251,0,0.00,NaT,...,0.0,0.0,0,3502232,1,0,None,NaN,2025-04-15 18:25:08.507613,WB-Realization-API


In [131]:


# Define the table name
table_name = 'wb_realization_reports'

# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                   # Your ClickHouse username
    password=password,                                  # Your ClickHouse password
    database='user1',                                   # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification
)

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
logging.info("Data inserted successfully!")

TypeError: object of type 'float' has no len()